# =============================
# BÀI TẬP 5: Ứng dụng trong y tế - Khuyến nghị phác đồ điều trị dựa trên luật kết hợp
# =============================

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Tạo dữ liệu giả lập
np.random.seed(42)

# Tạo thông tin bệnh nhân
n_patients = 300
patient_ids = [f'P{i}' for i in range(1, n_patients+1)]
ages = np.random.randint(18, 90, size=n_patients)
genders = np.random.choice(['Nam', 'Nữ'], size=n_patients)
conditions = np.random.choice(['Tim mạch', 'Hô hấp', 'Tiêu hóa', 'Thần kinh', 'Nội tiết'],
                             size=n_patients)

patients = pd.DataFrame({
    'PatientID': patient_ids,
    'Age': ages,
    'Gender': genders,
    'Condition': conditions
})

# Tạo danh sách phương pháp điều trị
treatments = [f'T{i}' for i in range(1, 51)]
# Tần suất sử dụng các phương pháp điều trị (ví dụ)
treatment_frequencies = {
    'Tim mạch': {t: 0.2 for t in treatments[:10]},
    'Hô hấp': {t: 0.15 for t in treatments[10:20]},
    'Tiêu hóa': {t: 0.25 for t in treatments[20:30]},
    'Thần kinh': {t: 0.18 for t in treatments[30:40]},
    'Nội tiết': {t: 0.22 for t in treatments[40:50]}
}

# Mối quan hệ giữa bệnh lý và kết quả điều trị (ví dụ)
condition_outcome_probs = {
    'Tim mạch': {'T1': 0.7, 'T2': 0.6, 'T3': 0.8, 'T4': 0.5, 'T5': 0.7, 'T6': 0.6, 'T7': 0.9, 'T8': 0.8, 'T9': 0.7, 'T10': 0.6},
    'Hô hấp': {'T11': 0.65, 'T12': 0.75, 'T13': 0.85, 'T14': 0.55, 'T15': 0.65, 'T16': 0.75, 'T17': 0.85, 'T18': 0.9, 'T19': 0.6, 'T20': 0.7},
    'Tiêu hóa': {'T21': 0.8, 'T22': 0.7, 'T23': 0.6, 'T24': 0.9, 'T25': 0.8, 'T26': 0.7, 'T27': 0.6, 'T28': 0.9, 'T29': 0.8, 'T30': 0.7},
    'Thần kinh': {'T31': 0.5, 'T32': 0.6, 'T33': 0.7, 'T34': 0.8, 'T35': 0.9, 'T36': 0.55, 'T37': 0.65, 'T38': 0.75, 'T39': 0.85, 'T40': 0.95},
    'Nội tiết': {'T41': 0.75, 'T42': 0.85, 'T43': 0.65, 'T44': 0.75, 'T45': 0.85, 'T46': 0.9, 'T47': 0.8, 'T48': 0.7, 'T49': 0.6, 'T50': 0.95}
}

In [3]:
# TODO: Tạo lịch sử điều trị cho mỗi bệnh nhân
# 1. Mỗi bệnh nhân có 3-10 phương pháp điều trị
treatment_history = {}
treatment_outcomes = {}  # Thêm dictionary này
for i, patient in enumerate(patient_ids):
    condition = patients.loc[i, 'Condition']
    available_treatments = list(condition_outcome_probs[condition].keys())
    num_treatments = np.random.randint(3, 8)  # Giảm số lượng treatments
    patient_treatments = np.random.choice(available_treatments, size=num_treatments, replace=False)
    treatment_history[patient] = patient_treatments

    # Gán kết quả điều trị dựa trên xác suất
    outcomes = []
    for treatment in patient_treatments:
        prob_success = condition_outcome_probs[condition][treatment]
        outcome = 'Thành công' if np.random.rand() < prob_success else 'Thất bại'
        outcomes.append(outcome)
    treatment_outcomes[patient] = list(zip(patient_treatments, outcomes))


# In ra một vài ví dụ
print("Ví dụ về lịch sử điều trị:")
for i, patient in enumerate(list(treatment_history.keys())[:3]):
    print(f"- Bệnh nhân {patient}: {treatment_history[patient]}")

print("\nVí dụ về kết quả điều trị:")
for i, patient in enumerate(list(treatment_outcomes.keys())[:3]):
    print(f"- Bệnh nhân {patient}: {treatment_outcomes[patient]}")

Ví dụ về lịch sử điều trị:
- Bệnh nhân P1: ['T45' 'T48' 'T43']
- Bệnh nhân P2: ['T33' 'T34' 'T39' 'T32']
- Bệnh nhân P3: ['T41' 'T42' 'T48']

Ví dụ về kết quả điều trị:
- Bệnh nhân P1: [(np.str_('T45'), 'Thành công'), (np.str_('T48'), 'Thành công'), (np.str_('T43'), 'Thành công')]
- Bệnh nhân P2: [(np.str_('T33'), 'Thành công'), (np.str_('T34'), 'Thành công'), (np.str_('T39'), 'Thất bại'), (np.str_('T32'), 'Thành công')]
- Bệnh nhân P3: [(np.str_('T41'), 'Thành công'), (np.str_('T42'), 'Thất bại'), (np.str_('T48'), 'Thất bại')]


In [4]:
# 2. Phương pháp điều trị phụ thuộc vào bệnh lý
condition_treatments = {
    'Tim mạch': treatments[:10],
    'Hô hấp': treatments[10:20],
    'Tiêu hóa': treatments[20:30],
    'Thần kinh': treatments[30:40],
    'Nội tiết': treatments[40:50]
}
for patient, condition in zip(patient_ids, patients['Condition']):
    available_treatments = condition_treatments[condition]
    num_treatments = len(treatment_history[patient])
    treatment_history[patient] = np.random.choice(available_treatments, size=num_treatments, replace=True)

# In ra lịch sử điều trị sau khi cập nhật theo bệnh lý
print("\nLịch sử điều trị sau khi phụ thuộc vào bệnh lý:")
for i, patient in enumerate(list(treatment_history.keys())[:5]):
    print(f"- Bệnh nhân {patient}: {treatment_history[patient]}")


Lịch sử điều trị sau khi phụ thuộc vào bệnh lý:
- Bệnh nhân P1: ['T41' 'T45' 'T43']
- Bệnh nhân P2: ['T31' 'T33' 'T36' 'T37']
- Bệnh nhân P3: ['T48' 'T44' 'T43']
- Bệnh nhân P4: ['T5' 'T9' 'T6']
- Bệnh nhân P5: ['T49' 'T46' 'T42' 'T45']


In [5]:
# 3. Gán kết quả điều trị (thành công/thất bại)
treatment_outcomes = {}
for patient in patient_ids:
    patient_treatments = treatment_history[patient]
    outcomes = np.random.choice(['Thành công', 'Thất bại'], size=len(patient_treatments))
    treatment_outcomes[patient] = list(zip(patient_treatments, outcomes))

# In ra một vài ví dụ về kết quả điều trị
print("\nVí dụ về kết quả điều trị:")
for i, patient in enumerate(list(treatment_outcomes.keys())[:5]):
    print(f"- Bệnh nhân {patient}: {treatment_outcomes[patient]}")

# Chuyển đổi dữ liệu lịch sử điều trị thành DataFrame
treatment_history_list = []
for patient, treatments in treatment_history.items():
    treatment_history_list.append(list(treatments))

# In ra danh sách lịch sử điều trị
print("\nDanh sách lịch sử điều trị:")
print(treatment_history_list[:5])  # In 5 danh sách đầu tiên


Ví dụ về kết quả điều trị:
- Bệnh nhân P1: [(np.str_('T41'), np.str_('Thành công')), (np.str_('T45'), np.str_('Thành công')), (np.str_('T43'), np.str_('Thành công'))]
- Bệnh nhân P2: [(np.str_('T31'), np.str_('Thành công')), (np.str_('T33'), np.str_('Thành công')), (np.str_('T36'), np.str_('Thành công')), (np.str_('T37'), np.str_('Thất bại'))]
- Bệnh nhân P3: [(np.str_('T48'), np.str_('Thành công')), (np.str_('T44'), np.str_('Thành công')), (np.str_('T43'), np.str_('Thất bại'))]
- Bệnh nhân P4: [(np.str_('T5'), np.str_('Thành công')), (np.str_('T9'), np.str_('Thất bại')), (np.str_('T6'), np.str_('Thành công'))]
- Bệnh nhân P5: [(np.str_('T49'), np.str_('Thất bại')), (np.str_('T46'), np.str_('Thất bại')), (np.str_('T42'), np.str_('Thành công')), (np.str_('T45'), np.str_('Thất bại'))]

Danh sách lịch sử điều trị:
[[np.str_('T41'), np.str_('T45'), np.str_('T43')], [np.str_('T31'), np.str_('T33'), np.str_('T36'), np.str_('T37')], [np.str_('T48'), np.str_('T44'), np.str_('T43')], [np.str_(

In [6]:
# TODO: Chuyển đổi lịch sử điều trị thành dữ liệu giao dịch
# 1. Mỗi bệnh nhân là một giao dịch
treatment_history_list = []
for patient, treatments in treatment_history.items():  # Giả sử treatment_history đã được tạo ở trên
    treatment_history_list.append(list(treatments))

# In ra một vài ví dụ về treatment_history_list
print("\nVí dụ về treatment_history_list (5 giao dịch đầu):")
for i, transaction in enumerate(treatment_history_list[:5]):
    print(f"- Giao dịch {i+1}: {transaction}")



Ví dụ về treatment_history_list (5 giao dịch đầu):
- Giao dịch 1: [np.str_('T41'), np.str_('T45'), np.str_('T43')]
- Giao dịch 2: [np.str_('T31'), np.str_('T33'), np.str_('T36'), np.str_('T37')]
- Giao dịch 3: [np.str_('T48'), np.str_('T44'), np.str_('T43')]
- Giao dịch 4: [np.str_('T5'), np.str_('T9'), np.str_('T6')]
- Giao dịch 5: [np.str_('T49'), np.str_('T46'), np.str_('T42'), np.str_('T45')]


In [7]:
# 2. Các phương pháp điều trị là các mục
te = TransactionEncoder()
te_ary = te.fit(treatment_history_list).transform(treatment_history_list)
transaction_df = pd.DataFrame(te_ary, columns=te.columns_)

# In ra DataFrame giao dịch
print("\nDataFrame giao dịch (10 hàng đầu):")
print(transaction_df.head(10))

# In ra thông tin về shape của DataFrame
print("\nShape của DataFrame giao dịch:", transaction_df.shape)


DataFrame giao dịch (10 hàng đầu):
      T1    T10    T11    T12    T13    T14    T15    T16    T17    T18  ...  \
0  False  False  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False  False  False  False  False  False  False  ...   
2  False  False  False  False  False  False  False  False  False  False  ...   
3  False  False  False  False  False  False  False  False  False  False  ...   
4  False  False  False  False  False  False  False  False  False  False  ...   
5  False  False  False  False  False  False  False  False  False  False  ...   
6  False  False  False  False  False  False  False   True   True   True  ...   
7  False  False   True  False   True   True   True  False  False  False  ...   
8  False  False   True   True  False   True  False   True  False  False  ...   
9  False  False  False  False  False  False  False  False  False  False  ...   

     T46    T47    T48    T49     T5    T50     T6     T7     T8     T9  
0  False 

In [8]:
# TODO: Áp dụng thuật toán Apriori và tạo luật kết hợp
# 1. Tìm các mẫu hình điều trị phổ biến
frequent_itemsets = apriori(transaction_df, min_support=0.01, use_colnames=True)

# In ra các tập phổ biến
print("\nCác tập phổ biến (10 hàng đầu):")
print(frequent_itemsets.head(10))




Các tập phổ biến (10 hàng đầu):
    support itemsets
0  0.086667     (T1)
1  0.090000    (T10)
2  0.086667    (T11)
3  0.083333    (T12)
4  0.066667    (T13)
5  0.076667    (T14)
6  0.080000    (T15)
7  0.090000    (T16)
8  0.080000    (T17)
9  0.080000    (T18)


In [9]:
# 2. Tạo luật kết hợp
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# In ra các luật kết hợp
print("\nCác luật kết hợp (10 hàng đầu):")
print(rules.head(10))


Các luật kết hợp (10 hàng đầu):
  antecedents consequents  antecedent support  consequent support   support  \
0        (T1)       (T10)            0.086667            0.090000  0.050000   
1       (T10)        (T1)            0.090000            0.086667  0.050000   
2        (T1)        (T2)            0.086667            0.080000  0.030000   
3        (T2)        (T1)            0.080000            0.086667  0.030000   
4        (T1)        (T3)            0.086667            0.063333  0.026667   
5        (T3)        (T1)            0.063333            0.086667  0.026667   
6        (T1)        (T4)            0.086667            0.080000  0.026667   
7        (T4)        (T1)            0.080000            0.086667  0.026667   
8        (T1)        (T5)            0.086667            0.073333  0.043333   
9        (T5)        (T1)            0.073333            0.086667  0.043333   

   confidence      lift  representativity  leverage  conviction  \
0    0.576923  6.410256       

In [10]:
# 3. Lọc các luật dựa trên kết quả điều trị
successful_treatments = {}
for patient, outcomes in treatment_outcomes.items():
    successful_treatments[patient] = [treatment for treatment, outcome in outcomes if outcome == 'Thành công']

filtered_rules = rules.copy()  # Ban đầu giữ lại tất cả các luật
for i, rule in rules.iterrows():
    antecedents = set(rule['antecedents'])
    successful_antecedent_counts = 0
    total_antecedent_occurrences = 0
    for patient, treatments in treatment_history.items():
        if antecedents.issubset(treatments):
            total_antecedent_occurrences += 1
            if all(treatment in successful_treatments.get(patient, []) for treatment in antecedents):
                successful_antecedent_counts += 1
    # Lọc luật nếu tỷ lệ thành công của antecedent dưới một ngưỡng nhất định (ví dụ: 0.5)
    if total_antecedent_occurrences > 0 and successful_antecedent_counts / total_antecedent_occurrences < 0.5:
        filtered_rules.drop(i, inplace=True, errors='ignore')

# In ra các luật sau khi lọc
print("\nCác luật kết hợp sau khi lọc (10 hàng đầu):")
print(filtered_rules.head(10))

# In ra shape của luật trước và sau khi lọc
print("\nSố lượng luật trước khi lọc:", rules.shape[0])
print("Số lượng luật sau khi lọc:", filtered_rules.shape[0])


Các luật kết hợp sau khi lọc (10 hàng đầu):
   antecedents consequents  antecedent support  consequent support   support  \
0         (T1)       (T10)            0.086667            0.090000  0.050000   
1        (T10)        (T1)            0.090000            0.086667  0.050000   
2         (T1)        (T2)            0.086667            0.080000  0.030000   
4         (T1)        (T3)            0.086667            0.063333  0.026667   
5         (T3)        (T1)            0.063333            0.086667  0.026667   
6         (T1)        (T4)            0.086667            0.080000  0.026667   
8         (T1)        (T5)            0.086667            0.073333  0.043333   
9         (T5)        (T1)            0.073333            0.086667  0.043333   
10        (T1)        (T6)            0.086667            0.073333  0.023333   
11        (T6)        (T1)            0.073333            0.086667  0.023333   

    confidence      lift  representativity  leverage  conviction  \
0     

In [11]:
# TODO: Xây dựng hệ khuyến nghị phác đồ điều trị
# def recommend_treatment(patient_treatments, rules, top_n=5):
#     recommendations = []
#     for _, rule in rules.iterrows():
#         antecedents = set(rule['antecedents'])
#         if antecedents.issubset(patient_treatments):
#             for treatment in rule['consequents']:
#                 if treatment not in patient_treatments:
#                     recommendations.append((treatment, rule['confidence'], rule['lift']))
#
#     # Sắp xếp theo độ tin cậy và độ nâng
#     recommendations.sort(key=lambda x: (x[1], x[2]), reverse=True)
#     return recommendations[:top_n]
def recommend_treatment(patient_treatments, rules, top_n=5):
    recommendations = []
    for _, rule in rules.iterrows():
        antecedents = set(rule['antecedents'])
        if antecedents.issubset(patient_treatments):
            for treatment in rule['consequents']:
                if treatment not in patient_treatments:
                    recommendations.append((treatment, rule['confidence'], rule['lift']))

    # Sắp xếp theo độ tin cậy và độ nâng
    recommendations.sort(key=lambda x: (x[1], x[2]), reverse=True)
    return recommendations[:top_n]

# Ví dụ sử dụng hệ khuyến nghị
patient_id_to_recommend = 'P1'
treatments_for_patient = treatment_history[patient_id_to_recommend]

# Kiểm tra xem có luật nào không
if filtered_rules.shape[0] == 0:
    print("\nKhông có luật kết hợp nào được tạo ra sau khi lọc. Cần xem xét lại các tham số Apriori hoặc dữ liệu.")
else:
    print(f"\nSố lượng luật kết hợp sau khi lọc: {filtered_rules.shape[0]}")
    # In ra một vài luật có lift cao nhất (ví dụ)
    top_lift_rules = filtered_rules.nlargest(5, 'lift')
    print("\nCác luật có lift cao nhất (5 luật đầu):")
    print(top_lift_rules[['antecedents', 'consequents', 'lift']].to_string())

    top_recommendations = recommend_treatment(treatments_for_patient, filtered_rules)

    if len(top_recommendations) > 0:
        print(f"\nTop {len(top_recommendations)} Recommendations for Patient {patient_id_to_recommend}:")
        for treatment, confidence, lift in top_recommendations:
            print(f"- Treatment: {treatment}, Confidence: {confidence:.2f}, Lift: {lift:.2f}")
    else:
        print(f"\nKhông có khuyến nghị nào cho bệnh nhân {patient_id_to_recommend} dựa trên các luật hiện có.")




Số lượng luật kết hợp sau khi lọc: 1574

Các luật có lift cao nhất (5 luật đầu):
          antecedents      consequents       lift
3130            (T34)  (T33, T31, T38)  18.750000
3000       (T28, T26)       (T25, T23)  17.142857
2920            (T22)  (T25, T27, T24)  15.789474
3216  (T40, T36, T39)            (T35)  15.789474
3229            (T35)  (T40, T36, T39)  15.789474

Top 5 Recommendations for Patient P1:
- Treatment: T47, Confidence: 0.67, Lift: 5.00
- Treatment: T50, Confidence: 0.58, Lift: 5.83
- Treatment: T47, Confidence: 0.52, Lift: 3.89
- Treatment: T47, Confidence: 0.43, Lift: 3.21
- Treatment: T50, Confidence: 0.42, Lift: 6.25


In [12]:
# TODO: Đánh giá chất lượng khuyến nghị và phân tích kết quả
def evaluate_recommendations(tracking_data, all_patient_ids, all_treatments):
    total_recommendations = 0
    applied_count = 0
    successful_count = 0
    failed_count = 0
    not_applied_count = 0
    unique_treatment_sets = set()  # Để theo dõi độ đa dạng
    patient_with_recommendation_count = 0 # Đếm số bệnh nhân có khuyến nghị

    for patient, recommendations in tracking_data.items():
        total_recommendations += len(recommendations)
        if recommendations: # Kiểm tra xem có khuyến nghị cho bệnh nhân này không
            patient_with_recommendation_count += 1
        for treatment, data in recommendations.items():
            if data['status'] == 'Áp dụng':
                applied_count += 1
                if data['outcome'] == 'Thành công':
                    successful_count += 1
                elif data['outcome'] == 'Thất bại':
                    failed_count += 1
            elif data['status'] == 'Không áp dụng':
                not_applied_count += 1
            unique_treatment_sets.add(frozenset(recommendations.keys())) # Đóng băng set để hashable

    accuracy = (successful_count / applied_count) * 100 if applied_count > 0 else 0
    application_rate = (applied_count / total_recommendations) * 100 if total_recommendations > 0 else 0
    success_rate = (successful_count / total_recommendations) * 100 if total_recommendations > 0 else 0
    failure_rate = (failed_count / total_recommendations) * 100 if total_recommendations > 0 else 0
    not_applied_rate = (not_applied_count / total_recommendations) * 100 if total_recommendations > 0 else 0
    coverage = (patient_with_recommendation_count / len(all_patient_ids)) * 100 if all_patient_ids else 0
    diversity = len(unique_treatment_sets)

    # Tính độ mới lạ (ví dụ đơn giản: đếm số lượng khuyến nghị không phổ biến)
    # (Cần dữ liệu về tần suất sử dụng các phương pháp điều trị trong thực tế)
    novel_recommendations_count = 0
    for patient, recommendations in tracking_data.items():
        for treatment, data in recommendations.items():
            # Giả sử có một hàm is_common_treatment(treatment)
            # if not is_common_treatment(treatment):
            #     novel_recommendations_count += 1
            pass # Tạm thời bỏ qua phần này vì cần dữ liệu thực tế

    novelty_rate = (novel_recommendations_count / total_recommendations) * 100 if total_recommendations > 0 else 0

    return {
        'accuracy': accuracy,
        'application_rate': application_rate,
        'success_rate': success_rate,
        'failure_rate': failure_rate,
        'not_applied_rate': not_applied_rate,
        'coverage': coverage,
        'diversity': diversity,
        'novelty_rate': novelty_rate
    }

# Dữ liệu theo dõi (ví dụ)
recommendation_tracking = {
    'P1': {
        'T12': {'status': 'Áp dụng', 'outcome': 'Thành công'},
        'T25': {'status': 'Không áp dụng', 'outcome': None},
        'T31': {'status': 'Áp dụng', 'outcome': 'Thất bại'}
    },
    'P2': {
        'T1': {'status': 'Áp dụng', 'outcome': 'Thành công'},
        'T18': {'status': 'Áp dụng', 'outcome': 'Thất bại'},
        'T33': {'status': 'Không áp dụng', 'outcome': None},
        'T42': {'status': 'Áp dụng', 'outcome': 'Thành công'}
    },
    'P3': {
        'T5': {'status': 'Áp dụng', 'outcome': 'Thành công'},
        'T9': {'status': 'Không áp dụng', 'outcome': None}
    },
    # ... Thêm dữ liệu cho các bệnh nhân khác
}

# Giả sử có danh sách tất cả các PatientID và Treatment
all_patient_ids = [f'P{i}' for i in range(1, 301)]  # Thay 301 bằng số lượng bệnh nhân thực tế
all_treatments = [f'T{i}' for i in range(1, 51)]   # Thay 51 bằng số lượng phương pháp điều trị thực tế

metrics = evaluate_recommendations(recommendation_tracking, all_patient_ids, all_treatments)

print("\nĐánh giá chất lượng khuyến nghị:")
print(f"  - Độ chính xác (Accuracy): {metrics['accuracy']:.2f}%")
print(f"  - Tỷ lệ áp dụng (Application Rate): {metrics['application_rate']:.2f}%")
print(f"  - Tỷ lệ thành công (Success Rate): {metrics['success_rate']:.2f}%")
print(f"  - Tỷ lệ thất bại (Failure Rate): {metrics['failure_rate']:.2f}%")
print(f"  - Tỷ lệ không áp dụng (Not Applied Rate): {metrics['not_applied_rate']:.2f}%")
print(f"  - Độ bao phủ (Coverage): {metrics['coverage']:.2f}%")
print(f"  - Độ đa dạng (Diversity): {metrics['diversity']}")
print(f"  - Tỷ lệ mới lạ (Novelty Rate): {metrics['novelty_rate']:.2f}%")


Đánh giá chất lượng khuyến nghị:
  - Độ chính xác (Accuracy): 66.67%
  - Tỷ lệ áp dụng (Application Rate): 66.67%
  - Tỷ lệ thành công (Success Rate): 44.44%
  - Tỷ lệ thất bại (Failure Rate): 22.22%
  - Tỷ lệ không áp dụng (Not Applied Rate): 33.33%
  - Độ bao phủ (Coverage): 1.00%
  - Độ đa dạng (Diversity): 3
  - Tỷ lệ mới lạ (Novelty Rate): 0.00%
